We will analyze the real estate market structure in Canada with the help of affinity propagation, an unsupervised clustering technique,
where we group local real estate markets that possess similar historical price fluctuations, the final result will be a market map revealing zones of strength and more risky areas as real estate investments.

Data was downloaded on the Canadian Gov website: https://open.canada.ca/data/en.
We also downloaded data from https://stats.crea.ca/en-CA/, https://www.crea.ca/housing-market-stats/mls-home-price-index/
where HPI -home price index- data is available. According to StatCan website, Home Price Index is the most advanced and accurate tool
to gauge home prices and trends.

(All this data might be used in the near future for a ML prediction bot of Canadian Real Estate price levels along with
unemployment, GDP growth, real interest rate, inflation % change, cost of materials/raw goods).

In [159]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn import cluster, covariance, manifold
import seaborn as sns

In [160]:
#After comparing all the csv downloaded, we notice that the file labeled a_18100202.csv downloaded here: 
# https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1810020502&pickMembers%5B0%5D=1.1&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=1982&referencePeriods=19820101%2C19820101
# does include more than 50000 lines with data from 1981 whereas the other files contain less than 1000 lines of data. 
# Therefore, this is the csv we will work with.

data = pd.read_csv(r'C:\Users\hp\Desktop\Projects Coding\Affinity_Propagation_Canada_Real_Estate_Market\Real Estate Data\a_18100205.csv')

#Previewing the data
data.head(20)

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1981-01,Canada,2016A000011124,Total (house and land),"Index, 201612=100",347,units,0,v111955442,1.1,38.2,NaN,NaN,NaN,1
1,1981-01,Canada,2016A000011124,House only,"Index, 201612=100",347,units,0,v111955443,1.2,36.1,NaN,NaN,NaN,1
2,1981-01,Canada,2016A000011124,Land only,"Index, 201612=100",347,units,0,v111955444,1.3,40.6,E,NaN,NaN,1
3,1981-01,Atlantic Region,2016A00011,Total (house and land),"Index, 201612=100",347,units,0,v111955445,2.1,NaN,..,NaN,NaN,1
4,1981-01,Atlantic Region,2016A00011,House only,"Index, 201612=100",347,units,0,v111955446,2.2,NaN,..,NaN,NaN,1
5,1981-01,Atlantic Region,2016A00011,Land only,"Index, 201612=100",347,units,0,v111955447,2.3,NaN,..,NaN,NaN,1
6,1981-01,Newfoundland and Labrador,2016A000210,Total (house and land),"Index, 201612=100",347,units,0,v111955448,3.1,NaN,..,NaN,NaN,1
7,1981-01,Newfoundland and Labrador,2016A000210,House only,"Index, 201612=100",347,units,0,v111955449,3.2,NaN,..,NaN,NaN,1
8,1981-01,Newfoundland and Labrador,2016A000210,Land only,"Index, 201612=100",347,units,0,v111955450,3.3,NaN,..,NaN,NaN,1
9,1981-01,"St. John's, Newfoundland and Labrador",2011S05031,Total (house and land),"Index, 201612=100",347,units,0,v111955451,4.1,36.1,NaN,NaN,NaN,1


In [161]:
"""
An index HPI is attributed to each region on a monthly basis since January 1981.
The GEO column includes string of specific Regions, cities or whole province, we extract unique values of this column here:
"""

#Extact unique string values in the GEO column
locations = data.GEO.unique()

#Converting the array to a list 
locations_list = locations.tolist()
print(locations_list)

['Canada', 'Atlantic Region', 'Newfoundland and Labrador', "St. John's, Newfoundland and Labrador", 'Prince Edward Island', 'Charlottetown, Prince Edward Island', 'Nova Scotia', 'Halifax, Nova Scotia', 'New Brunswick', 'Saint John, Fredericton, and Moncton, New Brunswick', 'Quebec', 'Québec, Quebec', 'Sherbrooke, Quebec', 'Trois-Rivières, Quebec', 'Montréal, Quebec', 'Ottawa-Gatineau, Quebec part, Ontario/Quebec', 'Ontario', 'Ottawa-Gatineau, Ontario part, Ontario/Quebec', 'Oshawa, Ontario', 'Toronto, Ontario', 'Hamilton, Ontario', 'St. Catharines-Niagara, Ontario', 'Kitchener-Cambridge-Waterloo, Ontario', 'Guelph, Ontario', 'London, Ontario', 'Windsor, Ontario', 'Greater Sudbury, Ontario', 'Prairie Region', 'Manitoba', 'Winnipeg, Manitoba', 'Saskatchewan', 'Regina, Saskatchewan', 'Saskatoon, Saskatchewan', 'Alberta', 'Calgary, Alberta', 'Edmonton, Alberta', 'British Columbia', 'Kelowna, British Columbia', 'Vancouver, British Columbia', 'Victoria, British Columbia']


In [162]:
#DGUID is Dissemination Geography Unique Identifier basically an identifier linking geospatial data with statistical data.
#Let's drop useless columns: STATUS	SYMBOL	TERMINATED	DECIMALS 

df = data.drop(columns=['SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE','STATUS','SYMBOL','TERMINATED','DECIMALS'], axis = 1)
df.head()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
0,1981-01,Canada,2016A000011124,Total (house and land),"Index, 201612=100",347,38.2
1,1981-01,Canada,2016A000011124,House only,"Index, 201612=100",347,36.1
2,1981-01,Canada,2016A000011124,Land only,"Index, 201612=100",347,40.6
3,1981-01,Atlantic Region,2016A00011,Total (house and land),"Index, 201612=100",347,NaN
4,1981-01,Atlantic Region,2016A00011,House only,"Index, 201612=100",347,NaN


In [163]:
#House Only Dataframe, as you may notice "New housing price indexes" includes three rows of data for each month, Total, House & Land
#We select only the rows including the "House only" data.

filter_list = ["House only"]
house_only_df = df[df["New housing price indexes"].isin(filter_list)]

# #We add '_' to Province names to simplify issues when filtering data 
# # to avoid confusion for example: Edmonton, Alberta VS Alberta
# house_only_df['GEO'] = house_only_df['GEO'].str.replace('Edmonton, Alberta', 'Edmonton, _Alberta')

house_only_df = house_only_df.reset_index()
#   house_only_df contains 20039 rows.
house_only_df.tail()

,index,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
20035,60106,2022-09,"Edmonton, Alberta",2011S0503835,House only,"Index, 201612=100",347,113.5
20036,60109,2022-09,British Columbia,2016A000259,House only,"Index, 201612=100",347,128.3
20037,60112,2022-09,"Kelowna, British Columbia",2011S0503915,House only,"Index, 201612=100",347,128.5
20038,60115,2022-09,"Vancouver, British Columbia",2011S0503933,House only,"Index, 201612=100",347,127.6
20039,60118,2022-09,"Victoria, British Columbia",2011S0503935,House only,"Index, 201612=100",347,132.1


We rely on the strings list extracted before to generate new dataframes for each region. 
The dataframes will subsequently be concatenated to aggregate all data.
List:  

'Atlantic Region', 'Newfoundland and Labrador', "St. John's, Newfoundland and Labrador", 'Prince Edward Island', 'Charlottetown, Prince Edward Island', 'Nova Scotia', 'Halifax, Nova Scotia', 'New Brunswick', 'Saint John, Fredericton, and Moncton, New Brunswick', 'Quebec', 'Québec, Quebec', 'Sherbrooke, Quebec', 'Trois-Rivières, Quebec', 'Montréal, Quebec', 'Ottawa-Gatineau, Quebec part, Ontario/Quebec', 'Ontario', 'Ottawa-Gatineau, Ontario part, Ontario/Quebec', 'Oshawa, Ontario', 'Toronto, Ontario', 'Hamilton, Ontario', 'St. Catharines-Niagara, Ontario', 'Kitchener-Cambridge-Waterloo, Ontario', 'Guelph, Ontario', 'London, Ontario', 'Windsor, Ontario', 'Greater Sudbury, Ontario', 'Prairie Region', 'Manitoba', 'Winnipeg, Manitoba', 'Saskatchewan', 'Regina, Saskatchewan', 'Saskatoon, Saskatchewan', 'Alberta', 'Calgary, Alberta', 'Edmonton, Alberta', 'British Columbia', 'Kelowna, British Columbia', 'Vancouver, British Columbia', 'Victoria, British Columbia'

In [164]:
#Filtering data of "Atlantic Region" and storing in atlantic_df
atlantic = house_only_df[house_only_df["GEO"].str.contains("Atlantic Region")]

atlantic.set_index("REF_DATE", inplace = True)
atlantic = atlantic.drop(columns=['index'], axis = 1)


atlantic = atlantic.reset_index()

#Saving df in 'Processed Data' folder
atlantic.to_csv('Processed Data/atlantic.csv')
#Previewing
atlantic.head()


,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
0,1981-01,Atlantic Region,2016A00011,House only,"Index, 201612=100",347,NaN
1,1981-02,Atlantic Region,2016A00011,House only,"Index, 201612=100",347,NaN
2,1981-03,Atlantic Region,2016A00011,House only,"Index, 201612=100",347,NaN
3,1981-04,Atlantic Region,2016A00011,House only,"Index, 201612=100",347,NaN
4,1981-05,Atlantic Region,2016A00011,House only,"Index, 201612=100",347,NaN


In [165]:
#Filtering data of 'Newfoundland and Labrador' 

nfland_labrador = house_only_df[house_only_df["GEO"].str.contains("St. John's, Newfoundland and Labrador")]
nfland_labrador.set_index("REF_DATE", inplace = True)
nfland_labrador = nfland_labrador.drop(columns=['index'], axis = 1)

#Index resetting
nfland_labrador = nfland_labrador.reset_index()

#Saving df in 'Processed Data' folder
nfland_lab.to_csv('Processed Data/nfland_labrador.csv')
#Previewing
nfland_labrador.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"St. John's, Newfoundland and Labrador",2011S05031,House only,"Index, 201612=100",347,106.7
497,2022-06,"St. John's, Newfoundland and Labrador",2011S05031,House only,"Index, 201612=100",347,107.4
498,2022-07,"St. John's, Newfoundland and Labrador",2011S05031,House only,"Index, 201612=100",347,107.4
499,2022-08,"St. John's, Newfoundland and Labrador",2011S05031,House only,"Index, 201612=100",347,107.4
500,2022-09,"St. John's, Newfoundland and Labrador",2011S05031,House only,"Index, 201612=100",347,108.1


In [166]:
'Prince Edward Island'

'Prince Edward Island'

In [167]:
#Filtering data of 'Charlottetown, Prince Edward Island' 

charlottetown_pei = house_only_df[house_only_df["GEO"].str.contains("Charlottetown, Prince Edward Island")]
charlottetown_pei.set_index("REF_DATE", inplace = True)
charlottetown_pei = charlottetown_pei.drop(columns=['index'], axis = 1)

#Index resetting
charlottetown_pei = charlottetown_pei.reset_index()

#Saving df in 'Processed Data' folder
charlottetown_pei.to_csv('Processed Data/nfland_labrador.csv')
#Previewing
charlottetown_pei.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Charlottetown, Prince Edward Island",2011S0504105,House only,"Index, 201612=100",347,125.5
497,2022-06,"Charlottetown, Prince Edward Island",2011S0504105,House only,"Index, 201612=100",347,126.9
498,2022-07,"Charlottetown, Prince Edward Island",2011S0504105,House only,"Index, 201612=100",347,127.3
499,2022-08,"Charlottetown, Prince Edward Island",2011S0504105,House only,"Index, 201612=100",347,127.3
500,2022-09,"Charlottetown, Prince Edward Island",2011S0504105,House only,"Index, 201612=100",347,127.0


In [168]:
'Nova Scotia' 

'Nova Scotia'

In [169]:
#Filtering data of 'Halifax, Nova Scotia' 

halifax_ns = house_only_df[house_only_df["GEO"].str.contains("Halifax, Nova Scotia")]
halifax_ns.set_index("REF_DATE", inplace = True)
halifax_ns = halifax_ns.drop(columns=['index'], axis = 1)

#Index resetting
halifax_ns = halifax_ns.reset_index()

#Saving df in 'Processed Data' folder
halifax_ns.to_csv('Processed Data/halifax_ns.csv')
#Previewing
halifax_ns.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Halifax, Nova Scotia",2011S0503205,House only,"Index, 201612=100",347,127.5
497,2022-06,"Halifax, Nova Scotia",2011S0503205,House only,"Index, 201612=100",347,127.5
498,2022-07,"Halifax, Nova Scotia",2011S0503205,House only,"Index, 201612=100",347,127.5
499,2022-08,"Halifax, Nova Scotia",2011S0503205,House only,"Index, 201612=100",347,127.5
500,2022-09,"Halifax, Nova Scotia",2011S0503205,House only,"Index, 201612=100",347,127.5


In [170]:
'New Brunswick'


'New Brunswick'

In [171]:
#Filtering data of 'Saint John, Fredericton, and Moncton, New Brunswick' 

stjohn_fredericton_moncton = house_only_df[house_only_df["GEO"].str.contains('Saint John, Fredericton, and Moncton, New Brunswick')]
stjohn_fredericton_moncton.set_index("REF_DATE", inplace = True)
stjohn_fredericton_moncton = stjohn_fredericton_moncton.drop(columns=['index'], axis = 1)

#Index resetting
stjohn_fredericton_moncton = stjohn_fredericton_moncton.reset_index()

#Saving df in 'Processed Data' folder
stjohn_fredericton_moncton.to_csv('Processed Data/stjohn_fredericton_moncton.csv')
#Previewing
stjohn_fredericton_moncton.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Saint John, Fredericton, and Moncton, New Brun...",NaN,House only,"Index, 201612=100",347,121.1
497,2022-06,"Saint John, Fredericton, and Moncton, New Brun...",NaN,House only,"Index, 201612=100",347,121.1
498,2022-07,"Saint John, Fredericton, and Moncton, New Brun...",NaN,House only,"Index, 201612=100",347,121.1
499,2022-08,"Saint John, Fredericton, and Moncton, New Brun...",NaN,House only,"Index, 201612=100",347,121.1
500,2022-09,"Saint John, Fredericton, and Moncton, New Brun...",NaN,House only,"Index, 201612=100",347,121.1


In [172]:
'Quebec' 


'Quebec'

In [202]:
#Filtering data of 'Québec, Quebec'

quebec_qc = house_only_df[house_only_df["GEO"].str.contains('Québec, Quebec')]
quebec_qc.set_index("REF_DATE", inplace = True)
quebec_qc = quebec_qc.drop(columns=['index'], axis = 1)

#Index resetting
quebec_qc = quebec_qc.reset_index()

#Saving df in 'Processed Data' folder
quebec_qc.to_csv('Processed Data/quebec_qc.csv')
#Previewing
quebec_qc.tail()



,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Québec, Quebec",2011S0503421,House only,"Index, 201612=100",347,132.6
497,2022-06,"Québec, Quebec",2011S0503421,House only,"Index, 201612=100",347,133.3
498,2022-07,"Québec, Quebec",2011S0503421,House only,"Index, 201612=100",347,133.3
499,2022-08,"Québec, Quebec",2011S0503421,House only,"Index, 201612=100",347,133.3
500,2022-09,"Québec, Quebec",2011S0503421,House only,"Index, 201612=100",347,133.3


In [203]:
#Filtering data of 'Sherbrooke, Quebec'

sherbrooke_qc = house_only_df[house_only_df["GEO"].str.contains('Sherbrooke, Quebec')]
sherbrooke_qc.set_index("REF_DATE", inplace = True)
sherbrooke_qc = sherbrooke_qc.drop(columns=['index'], axis = 1)

#Index resetting
sherbrooke_qc = sherbrooke_qc.reset_index()

#Saving df in 'Processed Data' folder
sherbrooke_qc.to_csv('Processed Data/sherbrooke_qc.csv')
#Previewing
sherbrooke_qc.tail()


,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Sherbrooke, Quebec",2011S0503433,House only,"Index, 201612=100",347,113.1
497,2022-06,"Sherbrooke, Quebec",2011S0503433,House only,"Index, 201612=100",347,113.1
498,2022-07,"Sherbrooke, Quebec",2011S0503433,House only,"Index, 201612=100",347,113.1
499,2022-08,"Sherbrooke, Quebec",2011S0503433,House only,"Index, 201612=100",347,113.1
500,2022-09,"Sherbrooke, Quebec",2011S0503433,House only,"Index, 201612=100",347,113.1


In [204]:
#Filtering data of 'Trois-Rivières, Quebec' 

troisriv_qc = house_only_df[house_only_df["GEO"].str.contains('Trois-Rivières, Quebec')]
troisriv_qc.set_index("REF_DATE", inplace = True)
troisriv_qc = troisriv_qc.drop(columns=['index'], axis = 1)

#Index resetting
troisriv_qc = troisriv_qc.reset_index()

#Saving df in 'Processed Data' folder
troisriv_qc.to_csv('Processed Data/troisriv_qc.csv')
#Previewing
troisriv_qc.tail()



,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Trois-Rivières, Quebec",2011S0503442,House only,"Index, 201612=100",347,112.6
497,2022-06,"Trois-Rivières, Quebec",2011S0503442,House only,"Index, 201612=100",347,112.6
498,2022-07,"Trois-Rivières, Quebec",2011S0503442,House only,"Index, 201612=100",347,112.6
499,2022-08,"Trois-Rivières, Quebec",2011S0503442,House only,"Index, 201612=100",347,112.6
500,2022-09,"Trois-Rivières, Quebec",2011S0503442,House only,"Index, 201612=100",347,112.6


In [206]:
#Filtering data of 'Montréal, Quebec'

mtl_qc = house_only_df[house_only_df["GEO"].str.contains('Montréal, Quebec')]
mtl_qc.set_index("REF_DATE", inplace = True)
mtl_qc = mtl_qc.drop(columns=['index'], axis = 1)

#Index resetting
mtl_qc = mtl_qc.reset_index()

#Saving df in 'Processed Data' folder
mtl_qc.to_csv('Processed Data/mtl_qc.csv')
#Previewing
mtl_qc.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Montréal, Quebec",2011S0503462,House only,"Index, 201612=100",347,161.1
497,2022-06,"Montréal, Quebec",2011S0503462,House only,"Index, 201612=100",347,160.9
498,2022-07,"Montréal, Quebec",2011S0503462,House only,"Index, 201612=100",347,160.6
499,2022-08,"Montréal, Quebec",2011S0503462,House only,"Index, 201612=100",347,160.7
500,2022-09,"Montréal, Quebec",2011S0503462,House only,"Index, 201612=100",347,160.6


In [209]:
#Filtering data of 'Ottawa-Gatineau, Quebec part, Ontario/Quebec'

ottawa_gatineau_qc = house_only_df[house_only_df["GEO"].str.contains('Ottawa-Gatineau, Quebec part, Ontario/Quebec')]
ottawa_gatineau_qc.set_index("REF_DATE", inplace = True)
ottawa_gatineau_qc = ottawa_gatineau_qc.drop(columns=['index'], axis = 1)

#Index resetting
ottawa_gatineau_qc = ottawa_gatineau_qc.reset_index()

#Saving df in 'Processed Data' folder
ottawa_gatineau_qc.to_csv('Processed Data/ottawa_gatineau_qc.csv')
#Previewing
ottawa_gatineau_qc.tail()


,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Ottawa-Gatineau, Quebec part, Ontario/Quebec",2011S050324505,House only,"Index, 201612=100",347,120.5
497,2022-06,"Ottawa-Gatineau, Quebec part, Ontario/Quebec",2011S050324505,House only,"Index, 201612=100",347,120.5
498,2022-07,"Ottawa-Gatineau, Quebec part, Ontario/Quebec",2011S050324505,House only,"Index, 201612=100",347,120.5
499,2022-08,"Ottawa-Gatineau, Quebec part, Ontario/Quebec",2011S050324505,House only,"Index, 201612=100",347,120.5
500,2022-09,"Ottawa-Gatineau, Quebec part, Ontario/Quebec",2011S050324505,House only,"Index, 201612=100",347,120.5


In [178]:
'Ontario'


'Ontario'

In [208]:
#Filtering data of 'Ottawa-Gatineau, Ontario part, Ontario/Quebec' 

ottawa_gatineau_ont = house_only_df[house_only_df["GEO"].str.contains('Ottawa-Gatineau, Ontario part, Ontario/Quebec')]
ottawa_gatineau_ont.set_index("REF_DATE", inplace = True)
ottawa_gatineau_ont = ottawa_gatineau_ont.drop(columns=['index'], axis = 1)

#Index resetting
ottawa_gatineau_ont = ottawa_gatineau_ont.reset_index()

#Saving df in 'Processed Data' folder
ottawa_gatineau_ont.to_csv('Processed Data/ottawa_gatineau_ont.csv')
#Previewing
ottawa_gatineau_ont.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Ottawa-Gatineau, Ontario part, Ontario/Quebec",2011S050335505,House only,"Index, 201612=100",347,187.4
497,2022-06,"Ottawa-Gatineau, Ontario part, Ontario/Quebec",2011S050335505,House only,"Index, 201612=100",347,188.9
498,2022-07,"Ottawa-Gatineau, Ontario part, Ontario/Quebec",2011S050335505,House only,"Index, 201612=100",347,190.2
499,2022-08,"Ottawa-Gatineau, Ontario part, Ontario/Quebec",2011S050335505,House only,"Index, 201612=100",347,189.4
500,2022-09,"Ottawa-Gatineau, Ontario part, Ontario/Quebec",2011S050335505,House only,"Index, 201612=100",347,189.4


In [210]:
#Filtering data of 'Oshawa, Ontario'

oshawa_ont = house_only_df[house_only_df["GEO"].str.contains('Oshawa, Ontario')]
oshawa_ont.set_index("REF_DATE", inplace = True)
oshawa_ont = oshawa_ont.drop(columns=['index'], axis = 1)

#Index resetting
oshawa_ont = oshawa_ont.reset_index()

#Saving df in 'Processed Data' folder
oshawa_ont.to_csv('Processed Data/oshawa_ont.csv')
#Previewing
oshawa_ont.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Oshawa, Ontario",2011S0503532,House only,"Index, 201612=100",347,125.3
497,2022-06,"Oshawa, Ontario",2011S0503532,House only,"Index, 201612=100",347,125.3
498,2022-07,"Oshawa, Ontario",2011S0503532,House only,"Index, 201612=100",347,125.3
499,2022-08,"Oshawa, Ontario",2011S0503532,House only,"Index, 201612=100",347,125.3
500,2022-09,"Oshawa, Ontario",2011S0503532,House only,"Index, 201612=100",347,125.3


In [211]:
#Filtering data of 'Toronto, Ontario' 

toronto_ont = house_only_df[house_only_df["GEO"].str.contains('Toronto, Ontario')]
toronto_ont.set_index("REF_DATE", inplace = True)
toronto_ont = toronto_ont.drop(columns=['index'], axis = 1)

#Index resetting
toronto_ont = toronto_ont.reset_index()

#Saving df in 'Processed Data' folder
toronto_ont.to_csv('Processed Data/oshawa_ont.csv')
#Previewing
toronto_ont.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Toronto, Ontario",2011S0503535,House only,"Index, 201612=100",347,114.6
497,2022-06,"Toronto, Ontario",2011S0503535,House only,"Index, 201612=100",347,114.6
498,2022-07,"Toronto, Ontario",2011S0503535,House only,"Index, 201612=100",347,114.6
499,2022-08,"Toronto, Ontario",2011S0503535,House only,"Index, 201612=100",347,114.6
500,2022-09,"Toronto, Ontario",2011S0503535,House only,"Index, 201612=100",347,114.6


In [213]:
#Filtering data of 'Hamilton, Ontario'

hamilton_ont = house_only_df[house_only_df["GEO"].str.contains('Hamilton, Ontario')]
hamilton_ont.set_index("REF_DATE", inplace = True)
hamilton_ont = hamilton_ont.drop(columns=['index'], axis = 1)

#Index resetting
hamilton_ont = hamilton_ont.reset_index()

#Saving df in 'Processed Data' folder
hamilton_ont.to_csv('Processed Data/hamilton_ont.csv')
#Previewing
hamilton_ont.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Hamilton, Ontario",2011S0503537,House only,"Index, 201612=100",347,121.9
497,2022-06,"Hamilton, Ontario",2011S0503537,House only,"Index, 201612=100",347,121.9
498,2022-07,"Hamilton, Ontario",2011S0503537,House only,"Index, 201612=100",347,121.9
499,2022-08,"Hamilton, Ontario",2011S0503537,House only,"Index, 201612=100",347,121.9
500,2022-09,"Hamilton, Ontario",2011S0503537,House only,"Index, 201612=100",347,121.3


In [214]:
#Filtering data of 'St. Catharines-Niagara, Ontario'

stcath_niagara_ont = house_only_df[house_only_df["GEO"].str.contains('St. Catharines-Niagara, Ontario')]
stcath_niagara_ont.set_index("REF_DATE", inplace = True)
stcath_niagara_ont = stcath_niagara_ont.drop(columns=['index'], axis = 1)

#Index resetting
stcath_niagara_ont = stcath_niagara_ont.reset_index()

#Saving df in 'Processed Data' folder
stcath_niagara_ont.to_csv('Processed Data/stcath_niagara_ont.csv')
#Previewing
stcath_niagara_ont.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"St. Catharines-Niagara, Ontario",2011S0503539,House only,"Index, 201612=100",347,133.0
497,2022-06,"St. Catharines-Niagara, Ontario",2011S0503539,House only,"Index, 201612=100",347,133.0
498,2022-07,"St. Catharines-Niagara, Ontario",2011S0503539,House only,"Index, 201612=100",347,133.0
499,2022-08,"St. Catharines-Niagara, Ontario",2011S0503539,House only,"Index, 201612=100",347,133.0
500,2022-09,"St. Catharines-Niagara, Ontario",2011S0503539,House only,"Index, 201612=100",347,132.7


In [217]:
#Filtering data of 'Kitchener-Cambridge-Waterloo, Ontario'

kitchener_camb_water_ont = house_only_df[house_only_df["GEO"].str.contains('Kitchener-Cambridge-Waterloo, Ontario')]
kitchener_camb_water_ont.set_index("REF_DATE", inplace = True)
kitchener_camb_water_ont = kitchener_camb_water_ont.drop(columns=['index'], axis = 1)

#Index resetting
kitchener_camb_water_ont = kitchener_camb_water_ont.reset_index()

#Saving df in 'Processed Data' folder
kitchener_camb_water_ont.to_csv('Processed Data/kitchener_camb_water_ont.csv')
#Previewing
kitchener_camb_water_ont.tail()


,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Kitchener-Cambridge-Waterloo, Ontario",2011S0503541,House only,"Index, 201612=100",347,154.3
497,2022-06,"Kitchener-Cambridge-Waterloo, Ontario",2011S0503541,House only,"Index, 201612=100",347,155.2
498,2022-07,"Kitchener-Cambridge-Waterloo, Ontario",2011S0503541,House only,"Index, 201612=100",347,154.8
499,2022-08,"Kitchener-Cambridge-Waterloo, Ontario",2011S0503541,House only,"Index, 201612=100",347,155.4
500,2022-09,"Kitchener-Cambridge-Waterloo, Ontario",2011S0503541,House only,"Index, 201612=100",347,154.7


In [219]:
#Filtering data of 'Guelph, Ontario' 

guelph_ont = house_only_df[house_only_df["GEO"].str.contains('Guelph, Ontario')]
guelph_ont.set_index("REF_DATE", inplace = True)
guelph_ont = guelph_ont.drop(columns=['index'], axis = 1)

#Index resetting
guelph_ont = guelph_ont.reset_index()

#Saving df in 'Processed Data' folder
guelph_ont.to_csv('Processed Data/guelph_ont.csv')
#Previewing
guelph_ont.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Guelph, Ontario",2011S0503550,House only,"Index, 201612=100",347,134.1
497,2022-06,"Guelph, Ontario",2011S0503550,House only,"Index, 201612=100",347,134.1
498,2022-07,"Guelph, Ontario",2011S0503550,House only,"Index, 201612=100",347,134.1
499,2022-08,"Guelph, Ontario",2011S0503550,House only,"Index, 201612=100",347,134.1
500,2022-09,"Guelph, Ontario",2011S0503550,House only,"Index, 201612=100",347,134.1


In [220]:
#Filtering data of 'London, Ontario'

london_ont = house_only_df[house_only_df["GEO"].str.contains('London, Ontario')]
london_ont.set_index("REF_DATE", inplace = True)
london_ont = london_ont.drop(columns=['index'], axis = 1)

#Index resetting
london_ont = london_ont.reset_index()

#Saving df in 'Processed Data' folder
london_ont.to_csv('Processed Data/london_ont.csv')
#Previewing
london_ont.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"London, Ontario",2011S0503555,House only,"Index, 201612=100",347,156.1
497,2022-06,"London, Ontario",2011S0503555,House only,"Index, 201612=100",347,156.1
498,2022-07,"London, Ontario",2011S0503555,House only,"Index, 201612=100",347,156.1
499,2022-08,"London, Ontario",2011S0503555,House only,"Index, 201612=100",347,156.1
500,2022-09,"London, Ontario",2011S0503555,House only,"Index, 201612=100",347,156.1


In [221]:
#Filtering data of 'Windsor, Ontario'

windsor_ont = house_only_df[house_only_df["GEO"].str.contains('Windsor, Ontario')]
windsor_ont.set_index("REF_DATE", inplace = True)
windsor_ont = windsor_ont.drop(columns=['index'], axis = 1)

#Index resetting
windsor_ont = windsor_ont.reset_index()

#Saving df in 'Processed Data' folder
windsor_ont.to_csv('Processed Data/windsor_ont.csv')
#Previewing
windsor_ont.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Windsor, Ontario",2011S0503559,House only,"Index, 201612=100",347,147.4
497,2022-06,"Windsor, Ontario",2011S0503559,House only,"Index, 201612=100",347,149.0
498,2022-07,"Windsor, Ontario",2011S0503559,House only,"Index, 201612=100",347,149.0
499,2022-08,"Windsor, Ontario",2011S0503559,House only,"Index, 201612=100",347,149.0
500,2022-09,"Windsor, Ontario",2011S0503559,House only,"Index, 201612=100",347,148.5


In [222]:
#Filtering data of 'Greater Sudbury, Ontario'

sudbury_ont = house_only_df[house_only_df["GEO"].str.contains('Greater Sudbury, Ontario')]
sudbury_ont.set_index("REF_DATE", inplace = True)
sudbury_ont = sudbury_ont.drop(columns=['index'], axis = 1)

#Index resetting
sudbury_ont = sudbury_ont.reset_index()

#Saving df in 'Processed Data' folder
sudbury_ont.to_csv('Processed Data/sudbury_ont.csv')
#Previewing
sudbury_ont.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Greater Sudbury, Ontario",2011S0503580,House only,"Index, 201612=100",347,122.3
497,2022-06,"Greater Sudbury, Ontario",2011S0503580,House only,"Index, 201612=100",347,122.5
498,2022-07,"Greater Sudbury, Ontario",2011S0503580,House only,"Index, 201612=100",347,122.5
499,2022-08,"Greater Sudbury, Ontario",2011S0503580,House only,"Index, 201612=100",347,122.5
500,2022-09,"Greater Sudbury, Ontario",2011S0503580,House only,"Index, 201612=100",347,122.5


In [223]:
#Filtering data of 'Prairie Region'

prairie_region = house_only_df[house_only_df["GEO"].str.contains('Prairie Region')]
prairie_region.set_index("REF_DATE", inplace = True)
prairie_region = prairie_region.drop(columns=['index'], axis = 1)

#Index resetting
prairie_region = prairie_region.reset_index()

#Saving df in 'Processed Data' folder
prairie_region.to_csv('Processed Data/prairie_region.csv')
#Previewing
prairie_region.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,Prairie Region,2016A00014,House only,"Index, 201612=100",347,126.1
497,2022-06,Prairie Region,2016A00014,House only,"Index, 201612=100",347,126.2
498,2022-07,Prairie Region,2016A00014,House only,"Index, 201612=100",347,125.9
499,2022-08,Prairie Region,2016A00014,House only,"Index, 201612=100",347,126.5
500,2022-09,Prairie Region,2016A00014,House only,"Index, 201612=100",347,126.4


In [190]:
'Manitoba'

'Manitoba'

In [224]:
#Filtering data of 'Winnipeg, Manitoba' 

winnipeg_manitoba = house_only_df[house_only_df["GEO"].str.contains('Winnipeg, Manitoba')]
winnipeg_manitoba.set_index("REF_DATE", inplace = True)
winnipeg_manitoba = winnipeg_manitoba.drop(columns=['index'], axis = 1)

#Index resetting
winnipeg_manitoba = winnipeg_manitoba.reset_index()

#Saving df in 'Processed Data' folder
winnipeg_manitoba.to_csv('Processed Data/winnipeg_manitoba.csv')
#Previewing
winnipeg_manitoba.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Winnipeg, Manitoba",2011S0503602,House only,"Index, 201612=100",347,158.6
497,2022-06,"Winnipeg, Manitoba",2011S0503602,House only,"Index, 201612=100",347,158.6
498,2022-07,"Winnipeg, Manitoba",2011S0503602,House only,"Index, 201612=100",347,158.6
499,2022-08,"Winnipeg, Manitoba",2011S0503602,House only,"Index, 201612=100",347,158.6
500,2022-09,"Winnipeg, Manitoba",2011S0503602,House only,"Index, 201612=100",347,158.6


In [192]:
'Saskatchewan'

'Saskatchewan'

In [225]:
#Filtering data of 'Regina, Saskatchewan'

regina_sask = house_only_df[house_only_df["GEO"].str.contains('Regina, Saskatchewan')]
regina_sask.set_index("REF_DATE", inplace = True)
regina_sask = regina_sask.drop(columns=['index'], axis = 1)

#Index resetting
regina_sask = regina_sask.reset_index()

#Saving df in 'Processed Data' folder
regina_sask.to_csv('Processed Data/regina_sask.csv')
#Previewing
regina_sask.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Regina, Saskatchewan",2011S0503705,House only,"Index, 201612=100",347,103.0
497,2022-06,"Regina, Saskatchewan",2011S0503705,House only,"Index, 201612=100",347,103.1
498,2022-07,"Regina, Saskatchewan",2011S0503705,House only,"Index, 201612=100",347,103.1
499,2022-08,"Regina, Saskatchewan",2011S0503705,House only,"Index, 201612=100",347,103.1
500,2022-09,"Regina, Saskatchewan",2011S0503705,House only,"Index, 201612=100",347,103.0


In [226]:
#Filtering data of 'Saskatoon, Saskatchewan'

saskatoon_sask = house_only_df[house_only_df["GEO"].str.contains('Saskatoon, Saskatchewan')]
saskatoon_sask.set_index("REF_DATE", inplace = True)
saskatoon_sask = saskatoon_sask.drop(columns=['index'], axis = 1)

#Index resetting
saskatoon_sask = saskatoon_sask.reset_index()

#Saving df in 'Processed Data' folder
saskatoon_sask.to_csv('Processed Data/saskatoon_sask.csv')
#Previewing
saskatoon_sask.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Saskatoon, Saskatchewan",2011S0503725,House only,"Index, 201612=100",347,110.0
497,2022-06,"Saskatoon, Saskatchewan",2011S0503725,House only,"Index, 201612=100",347,110.7
498,2022-07,"Saskatoon, Saskatchewan",2011S0503725,House only,"Index, 201612=100",347,110.5
499,2022-08,"Saskatoon, Saskatchewan",2011S0503725,House only,"Index, 201612=100",347,118.4
500,2022-09,"Saskatoon, Saskatchewan",2011S0503725,House only,"Index, 201612=100",347,118.3


In [195]:
'Alberta' 

'Alberta'

In [227]:
#Filtering data of 'Calgary, Alberta'

calgary_alb = house_only_df[house_only_df["GEO"].str.contains('Calgary, Alberta')]
calgary_alb.set_index("REF_DATE", inplace = True)
calgary_alb = calgary_alb.drop(columns=['index'], axis = 1)

#Index resetting
calgary_alb = calgary_alb.reset_index()

#Saving df in 'Processed Data' folder
calgary_alb.to_csv('Processed Data/calgary_alb.csv')
#Previewing
calgary_alb.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Calgary, Alberta",2011S0503825,House only,"Index, 201612=100",347,133.4
497,2022-06,"Calgary, Alberta",2011S0503825,House only,"Index, 201612=100",347,133.6
498,2022-07,"Calgary, Alberta",2011S0503825,House only,"Index, 201612=100",347,133.2
499,2022-08,"Calgary, Alberta",2011S0503825,House only,"Index, 201612=100",347,133.6
500,2022-09,"Calgary, Alberta",2011S0503825,House only,"Index, 201612=100",347,133.0


In [228]:
#Filtering data of 'Edmonton, Alberta'

edmonton_alb = house_only_df[house_only_df["GEO"].str.contains('Edmonton, Alberta')]
edmonton_alb.set_index("REF_DATE", inplace = True)
edmonton_alb = edmonton_alb.drop(columns=['index'], axis = 1)

#Index resetting
edmonton_alb = edmonton_alb.reset_index()

#Saving df in 'Processed Data' folder
edmonton_alb.to_csv('Processed Data/edmonton_alb.csv')
#Previewing
edmonton_alb.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Edmonton, Alberta",2011S0503835,House only,"Index, 201612=100",347,113.5
497,2022-06,"Edmonton, Alberta",2011S0503835,House only,"Index, 201612=100",347,113.6
498,2022-07,"Edmonton, Alberta",2011S0503835,House only,"Index, 201612=100",347,113.2
499,2022-08,"Edmonton, Alberta",2011S0503835,House only,"Index, 201612=100",347,113.4
500,2022-09,"Edmonton, Alberta",2011S0503835,House only,"Index, 201612=100",347,113.5


In [198]:
'British Columbia'

'British Columbia'

In [230]:
#Filtering data of 'Kelowna, British Columbia'

kelowna_bc = house_only_df[house_only_df["GEO"].str.contains('Kelowna, British Columbia')]
kelowna_bc.set_index("REF_DATE", inplace = True)
kelowna_bc = kelowna_bc.drop(columns=['index'], axis = 1)

#Index resetting
kelowna_bc = kelowna_bc.reset_index()

#Saving df in 'Processed Data' folder
kelowna_bc.to_csv('Processed Data/kelowna_bc.csv')
#Previewing
kelowna_bc.tail()


,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Kelowna, British Columbia",2011S0503915,House only,"Index, 201612=100",347,128.3
497,2022-06,"Kelowna, British Columbia",2011S0503915,House only,"Index, 201612=100",347,128.3
498,2022-07,"Kelowna, British Columbia",2011S0503915,House only,"Index, 201612=100",347,128.4
499,2022-08,"Kelowna, British Columbia",2011S0503915,House only,"Index, 201612=100",347,128.5
500,2022-09,"Kelowna, British Columbia",2011S0503915,House only,"Index, 201612=100",347,128.5


In [231]:
#Filtering data of 'Vancouver, British Columbia'

vancouver_bc = house_only_df[house_only_df["GEO"].str.contains('Vancouver, British Columbia')]
vancouver_bc.set_index("REF_DATE", inplace = True)
vancouver_bc = vancouver_bc.drop(columns=['index'], axis = 1)

#Index resetting
vancouver_bc = vancouver_bc.reset_index()

#Saving df in 'Processed Data' folder
vancouver_bc.to_csv('Processed Data/vancouver_bc.csv')
#Previewing
vancouver_bc.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Vancouver, British Columbia",2011S0503933,House only,"Index, 201612=100",347,127.2
497,2022-06,"Vancouver, British Columbia",2011S0503933,House only,"Index, 201612=100",347,127.2
498,2022-07,"Vancouver, British Columbia",2011S0503933,House only,"Index, 201612=100",347,127.6
499,2022-08,"Vancouver, British Columbia",2011S0503933,House only,"Index, 201612=100",347,127.6
500,2022-09,"Vancouver, British Columbia",2011S0503933,House only,"Index, 201612=100",347,127.6


In [232]:
#Filtering data of 'Victoria, British Columbia'

victoria_bc = house_only_df[house_only_df["GEO"].str.contains('Victoria, British Columbia')]
victoria_bc.set_index("REF_DATE", inplace = True)
victoria_bc = victoria_bc.drop(columns=['index'], axis = 1)

#Index resetting
victoria_bc = victoria_bc.reset_index()

#Saving df in 'Processed Data' folder
victoria_bc.to_csv('Processed Data/victoria_bc.csv')
#Previewing
victoria_bc.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,VALUE
496,2022-05,"Victoria, British Columbia",2011S0503935,House only,"Index, 201612=100",347,132.1
497,2022-06,"Victoria, British Columbia",2011S0503935,House only,"Index, 201612=100",347,132.1
498,2022-07,"Victoria, British Columbia",2011S0503935,House only,"Index, 201612=100",347,132.1
499,2022-08,"Victoria, British Columbia",2011S0503935,House only,"Index, 201612=100",347,132.1
500,2022-09,"Victoria, British Columbia",2011S0503935,House only,"Index, 201612=100",347,132.1


All data pertaining to each major Canadian city has been cleaned/processed in unique csv files under the 